In [1]:
#connect google colab with google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import f1_score
#from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import time
import sys
import math
#from os import listdir
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import InceptionV3

In [50]:
#initializing variables
BATCH_SIZE = 32
IMAGE_SIZE = 299
CHANNELS=3
EPOCHS=50
num_classes=2

In [72]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers

#input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

train_data_dir = '/content/gdrive/MyDrive/MajorProject/br35hdataset/train'
validation_data_dir = '/content/gdrive/MyDrive/MajorProject/br35hdataset/validation'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255    
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 14400 images belonging to 2 classes.
Found 1800 images belonging to 2 classes.


In [73]:
# import required libraries
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Define input shape
img_width, img_height = 299, 299
input_shape = (img_width, img_height, 3)

# Load InceptionV3 model with pre-trained weights from ImageNet
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in inception.layers:
    layer.trainable = False

In [74]:
model = Sequential()

# Add the InceptionV3 model as the first layer
model.add(inception)

# Add a Flatten layer to convert the output of the Inception V3 model to a 1D vector
#model.add(Flatten(input_shape=inception.output_shape[1:]))
model.add(Flatten())

# Add a Dense layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Add the final Dense layer with 2 units and softmax activation for binary classification
model.add(Dense(2, activation='softmax'))

# Reshape the output of the InceptionV3 model to match the input shape of the custom model
x = inception.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)

In [75]:
# concatenate the Inception V3 model with the custom model
model = Model(inputs=inception.input, outputs=predictions)

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# train the model
history = model.fit(
    train_generator,
    batch_size=BATCH_SIZE,
    validation_data=validation_generator,
    verbose=1,
    epochs=EPOCHS)

Epoch 1/50
  3/450 [..............................] - ETA: 1:15:29 - loss: 10.6954 - accuracy: 0.5312

In [ ]:
# Load the test folder
test_data_dir = '/content/gdrive/MyDrive/MajorProject/br35hdataset/test'

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# evaluate the model on the test set
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])